In [2]:
import pandas as pd
from pathlib import Path
import sqlite3
from IPython.display import display

HERE = Path.cwd()

def find_data_dir(start: Path) -> Path | None:
    for root in [start, *start.parents]:
        d = root / "data"
        if (d / "universe.sqlite").exists() or (d / "edges.parquet").exists():
            return d
    return None

DATA = find_data_dir(HERE)

display({"cwd": str(HERE), "data_dir": str(DATA) if DATA else None})

if DATA is None:
    raise FileNotFoundError(
        "Could not find a data directory.\n"
        "Expected either:\n"
        "  data/universe.sqlite  OR  data/edges.parquet\n"
        f"Starting from: {HERE}"
    )

def load_table(name: str) -> pd.DataFrame:
    parquet = DATA / f"{name}.parquet"
    sqlite_path = DATA / "universe.sqlite"

    if parquet.exists():
        try:
            return pd.read_parquet(parquet)
        except Exception as e:
            display({"parquet_read_failed": str(parquet), "error": str(e)})

    if sqlite_path.exists():
        with sqlite3.connect(sqlite_path) as conn:
            return pd.read_sql(f"SELECT * FROM {name}", conn)

    raise FileNotFoundError(f"Missing {name}.parquet and universe.sqlite in {DATA}")

equities = load_table("equities")
etfs     = load_table("etfs")
edges    = load_table("edges")

# Normalize tickers
if "symbol" in equities.columns:
    equities["symbol"] = equities["symbol"].astype(str).str.upper().str.strip()
if "src" in edges.columns:
    edges["src"] = edges["src"].astype(str).str.upper().str.strip()
if "dst" in edges.columns:
    edges["dst"] = edges["dst"].astype(str).str.upper().str.strip()

display({
    "equities_shape": equities.shape,
    "etfs_shape": etfs.shape,
    "edges_shape": edges.shape,
    "equities_cols": list(equities.columns),
    "etfs_cols": list(etfs.columns),
    "edges_cols": list(edges.columns),
})

display(equities.head(10))
display(etfs.head(10))
display(edges.head(10))


{'cwd': 'c:\\Users\\pwpat\\OneDrive\\Documents\\GitHub\\index_rel_qqq\\ndx_levered_etf_mapper\\notebooks',
 'data_dir': 'c:\\Users\\pwpat\\OneDrive\\Documents\\GitHub\\index_rel_qqq\\ndx_levered_etf_mapper\\data'}

{'equities_shape': (101, 4),
 'etfs_shape': (119, 7),
 'edges_shape': (119, 14),
 'equities_cols': ['symbol', 'company', 'sector', 'industry'],
 'etfs_cols': ['ticker',
  'name',
  'issuer',
  'asset_class',
  'strategy_group',
  'theme_tags',
  'source'],
 'edges_cols': ['src',
  'dst',
  'edge_type',
  'direction',
  'leverage_multiple',
  'relationship',
  'relationship_group',
  'daily_target',
  'issuer',
  'source_url',
  'etf_name',
  'strategy_group',
  'weight',
  'asof']}

,symbol,company,sector,industry
0,ADBE,Adobe Inc.,Computer Software,Technology
1,AMD,Advanced Micro Devices,Semiconductors,Technology
2,ABNB,Airbnb,Diversified Commercial Services,Consumer Discretionary
3,ALNY,Alnylam Pharmaceuticals,Biotechnology,Health Care
4,GOOGL,Alphabet Inc. (Class A),Computer Software,Technology
5,GOOG,Alphabet Inc. (Class C),Computer Software,Technology
6,AMZN,Amazon,Catalog/Specialty Distribution,Consumer Discretionary
7,AEP,American Electric Power,Electric Utilities,Utilities
8,AMGN,Amgen,Biotechnology,Health Care
9,ADI,Analog Devices,Semiconductors,Technology


,ticker,name,issuer,asset_class,strategy_group,theme_tags,source
0,STOCK,Direxion Daily AAPL Bull 2X,Direxion,etf,leveraged,None,https://www.direxion.com/uploads/Single-Stock-...
1,MUU,Direxion Daily MU Bull 2X,Direxion,etf,leveraged,None,https://www.direxion.com/uploads/Single-Stock-...
2,AAPD,Direxion Daily AAPL Bear 1X,Direxion,etf,inverse,None,https://www.direxion.com/uploads/Single-Stock-...
3,MUD,Direxion Daily MU Bear 1X,Direxion,etf,inverse,None,https://www.direxion.com/uploads/Single-Stock-...
4,MICRO,Direxion Daily AMD Bull 2X,Direxion,etf,leveraged,None,https://www.direxion.com/uploads/Single-Stock-...
5,MSFU,Direxion Daily MSFT Bull 2X,Direxion,etf,leveraged,None,https://www.direxion.com/uploads/Single-Stock-...
6,AMDD,Direxion Daily AMD Bear 1X,Direxion,etf,inverse,None,https://www.direxion.com/uploads/Single-Stock-...
7,MSFD,Direxion Daily MSFT Bear 1X,Direxion,etf,inverse,None,https://www.direxion.com/uploads/Single-Stock-...
8,AMZU,Direxion Daily AMZN Bull 2X,Direxion,etf,leveraged,None,https://www.direxion.com/uploads/Single-Stock-...
9,NVDU,Direxion Daily NVDA Bull 2X,Direxion,etf,leveraged,None,https://www.direxion.com/uploads/Single-Stock-...


,src,dst,edge_type,direction,leverage_multiple,relationship,relationship_group,daily_target,issuer,source_url,etf_name,strategy_group,weight,asof
0,AAPL,STOCK,derivative_exposure,long,2.0,leveraged_long,single_stock_leveraged,2.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily AAPL Bull 2X,leveraged,None,None
1,MU,MUU,derivative_exposure,long,2.0,leveraged_long,single_stock_leveraged,2.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily MU Bull 2X,leveraged,None,None
2,AAPL,AAPD,derivative_exposure,short,1.0,inverse,single_stock_inverse,-1.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily AAPL Bear 1X,inverse,None,None
3,MU,MUD,derivative_exposure,short,1.0,inverse,single_stock_inverse,-1.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily MU Bear 1X,inverse,None,None
4,AMD,MICRO,derivative_exposure,long,2.0,leveraged_long,single_stock_leveraged,2.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily AMD Bull 2X,leveraged,None,None
5,MSFT,MSFU,derivative_exposure,long,2.0,leveraged_long,single_stock_leveraged,2.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily MSFT Bull 2X,leveraged,None,None
6,AMD,AMDD,derivative_exposure,short,1.0,inverse,single_stock_inverse,-1.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily AMD Bear 1X,inverse,None,None
7,MSFT,MSFD,derivative_exposure,short,1.0,inverse,single_stock_inverse,-1.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily MSFT Bear 1X,inverse,None,None
8,AMZN,AMZU,derivative_exposure,long,2.0,leveraged_long,single_stock_leveraged,2.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily AMZN Bull 2X,leveraged,None,None
9,NVDA,NVDU,derivative_exposure,long,2.0,leveraged_long,single_stock_leveraged,2.0,Direxion,https://www.direxion.com/uploads/Single-Stock-...,Direxion Daily NVDA Bull 2X,leveraged,None,None


In [3]:
from IPython.display import display

edges_ndx = edges[edges["src"].isin(set(equities["symbol"]))].copy()

# Ensure types
edges_ndx["daily_target"] = pd.to_numeric(edges_ndx["daily_target"], errors="coerce")
edges_ndx["leverage_multiple"] = pd.to_numeric(edges_ndx["leverage_multiple"], errors="coerce")

# Human bucket
def bucket_row(r):
    et = r.get("edge_type", "")
    if et != "derivative_exposure":
        return "holds"

    sg = str(r.get("strategy_group", "") or "")
    direction = str(r.get("direction", "") or "")
    lev = r.get("leverage_multiple", None)

    if sg == "leveraged":
        # long/short is direction for derivative exposure
        if direction == "long":
            return "bull_levered"
        if direction == "short":
            return "bear_levered"
        return "levered_other"

    if sg == "inverse":
        return "inverse"

    return sg or "plain"

edges_ndx["bucket"] = edges_ndx.apply(bucket_row, axis=1)

def lev_label(x):
    if pd.isna(x):
        return ""
    # show as 2x, 3x, 1x
    if float(x).is_integer():
        return f"{int(x)}x"
    return f"{x:.2f}x"

edges_ndx["lev_label"] = edges_ndx["leverage_multiple"].apply(lev_label)

display(edges_ndx[["src","dst","issuer","bucket","lev_label","daily_target"]].head(20))


,src,dst,issuer,bucket,lev_label,daily_target
0,AAPL,STOCK,Direxion,bull_levered,2x,2.0
1,MU,MUU,Direxion,bull_levered,2x,2.0
2,AAPL,AAPD,Direxion,inverse,1x,-1.0
3,MU,MUD,Direxion,inverse,1x,-1.0
4,AMD,MICRO,Direxion,bull_levered,2x,2.0
5,MSFT,MSFU,Direxion,bull_levered,2x,2.0
6,AMD,AMDD,Direxion,inverse,1x,-1.0
7,MSFT,MSFD,Direxion,inverse,1x,-1.0
8,AMZN,AMZU,Direxion,bull_levered,2x,2.0
9,NVDA,NVDU,Direxion,bull_levered,2x,2.0


In [4]:
coverage = (
    edges_ndx
    .groupby("src")
    .agg(
        etf_count=("dst","nunique"),
        bull=("bucket", lambda s: (s=="bull_levered").sum()),
        bear=("bucket", lambda s: (s=="bear_levered").sum()),
        inverse=("bucket", lambda s: (s=="inverse").sum()),
        issuers=("issuer", lambda s: ", ".join(sorted(set([x for x in s.dropna().astype(str) if x])))),
    )
    .reset_index()
    .merge(equities[["symbol","company","sector"]], left_on="src", right_on="symbol", how="left")
    .drop(columns=["symbol"])
    .sort_values(["etf_count","src"], ascending=[False, True])
)

display(coverage.head(30))
display(coverage.tail(30))  # shows the names with 0 coverage once holdings provider exists; currently most will be 0


,src,etf_count,bull,bear,inverse,issuers,company,sector
15,TSLA,70,69,0,1,"Direxion, GraniteShares","Tesla, Inc.",Automobiles & Parts
0,AAPL,2,1,0,1,Direxion,Apple Inc.,Computer Manufacturing
1,AMD,2,1,0,1,Direxion,Advanced Micro Devices,Semiconductors
2,AMZN,2,1,0,1,Direxion,Amazon,Catalog/Specialty Distribution
3,AVGO,2,1,0,1,Direxion,Broadcom,Semiconductors
4,CSCO,2,1,0,1,Direxion,Cisco,Computer Communications Equipment
6,META,2,1,0,1,Direxion,Meta Platforms,Computer Software
7,MSFT,2,1,0,1,Direxion,Microsoft,Computer Software
8,MU,2,1,0,1,Direxion,Micron Technology,Semiconductors
9,NFLX,2,1,0,1,Direxion,"Netflix, Inc.",Consumer Electronics


,src,etf_count,bull,bear,inverse,issuers,company,sector
15,TSLA,70,69,0,1,"Direxion, GraniteShares","Tesla, Inc.",Automobiles & Parts
0,AAPL,2,1,0,1,Direxion,Apple Inc.,Computer Manufacturing
1,AMD,2,1,0,1,Direxion,Advanced Micro Devices,Semiconductors
2,AMZN,2,1,0,1,Direxion,Amazon,Catalog/Specialty Distribution
3,AVGO,2,1,0,1,Direxion,Broadcom,Semiconductors
4,CSCO,2,1,0,1,Direxion,Cisco,Computer Communications Equipment
6,META,2,1,0,1,Direxion,Meta Platforms,Computer Software
7,MSFT,2,1,0,1,Direxion,Microsoft,Computer Software
8,MU,2,1,0,1,Direxion,Micron Technology,Semiconductors
9,NFLX,2,1,0,1,Direxion,"Netflix, Inc.",Consumer Electronics


In [ ]:
all_coverage = equities[["symbol","company","sector"]].rename(columns={"symbol":"src"}).merge(
    coverage[["src","etf_count","bull","bear","inverse","issuers"]],
    on="src",
    how="left"
).fillna({"etf_count":0,"bull":0,"bear":0,"inverse":0,"issuers":""}).sort_values(["etf_count","src"], ascending=[False, True])

display(all_coverage.head(40))
